<a href="https://colab.research.google.com/github/ashwanth10/Deep_Learning/blob/master/assignment_2_part_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# To install wandb package ,which is helpful in generating plots and report.
!pip install wandb

In [ ]:
# Libraries used
import numpy as np

# Keras libraries for CNN
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras_preprocessing.image import ImageDataGenerator

# Import pre-trained models
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3, ResNet50, InceptionResNetV2, Xception

import os

import wandb
from wandb.keras import WandbCallback

In [ ]:
# Load Dataset if not available
if 'INaturalist_Dataset' not in os.listdir():
    !git clone https://github.com/ashwanth10/INaturalist_Dataset.git

# Create Model based on config values

In [ ]:
def build_model(config):

    # Clear the session
    tf.keras.backend.clear_session()
    
    # Based on pretrained model, create a base model
    if config.pretrained_model == 'resnet50':
        base_model = tf.keras.applications.ResNet152V2(weights="imagenet", input_shape=(300, 300, 3), 
        classes=1000, classifier_activation="softmax", include_top=False)
    elif config.pretrained_model == 'inceptionV3':
        base_model = tf.keras.applications.InceptionV3(weights="imagenet", input_shape=(300, 300, 3), 
        classes=1000, classifier_activation="softmax", include_top=False)
    elif config.pretrained_model == 'inceptionresnetv2':
        base_model = tf.keras.applications.InceptionResNetV2(weights="imagenet", input_shape=(300, 300, 3), 
        classes=1000, classifier_activation="softmax", include_top=False)
    elif config.pretrained_model == 'xception':
        base_model = tf.keras.applications.Xception(weights="imagenet", input_shape=(300, 300, 3), 
        classes=1000, classifier_activation="softmax", include_top=False)

    # Freeze layers in base model to compute faster
    total_no_of_layers_in_base_model = len(base_model.layers)
    base_model.trainable = False
    for i in range(total_no_of_layers_in_base_model - config.freeze_before, total_no_of_layers_in_base_model):
        base_model.layers[i].trainable = True

    # Create a new model using base model and dense layers
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.experimental.preprocessing.RandomCrop(height=300, width=300))
    model.add(tf.keras.Input(shape=(300, 300, 3)))
    
    # Add base model
    model.add(base_model)
    model.add(tf.keras.layers.Flatten())
    if config.no_of_neurons_in_dense:
        model.add(tf.keras.layers.Dense(1024, activation='relu'))
    
    # Add output layer
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    return model

# Sweep Config

In [ ]:
sweep_config = {
    'method': 'random', 
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'pretrained_model' :{
           'values' : ['resnet50','inceptionresnetv2', 'inceptionV3', 'xception']
        },
        'freeze_before' : {
            'values': [50,70,100]
        },
        'no_of_neurons_in_dense':{
            'values': [64, 128, 256, 512]
        },
        'epochs' : {
            'values': [2]
        },
    }
}

In [ ]:
# Initialize a new sweep
sweep_id = wandb.sweep(sweep_config, entity="cs21m010-cs21m041", project="DL_Assignment_2")

In [ ]:
def train():
    
    # Set seed
    np.random.seed(0)

    wandb.init(config=sweep_config)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    wandb.run.name = 'part_B_epochs_{}_base_model_{}_dense_{}_freeze_before_{}'.format(config.epochs, config.pretrained_model, config.no_of_neurons_in_dense, config.freeze_before)

    train_data = ImageDataGenerator(rescale=1. / 255).flow_from_directory('INaturalist_Dataset/train', shuffle=True, target_size=(400, 400))
    val_data = ImageDataGenerator(rescale=1. / 255).flow_from_directory('INaturalist_Dataset/val', shuffle=True, target_size=(400, 400))

    # Build model and train using a pre-trained model
    model = build_model(config)
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=["accuracy"])
    model.fit(train_data, steps_per_epoch=len(train_data), epochs=config.epochs, validation_data=val_data, callbacks=[WandbCallback()])

In [ ]:
wandb.agent(sweep_id, train, count = 20)